One of the applications of convex optimization in complex variables is phase recovery, a problem in signal processing. The objective is to reconstruct the complex phase of a vector when we are only given the magnitude of some linear measurements.

This problem is orginally a nonconvex optimization problem. Using results from [this paper](https://arxiv.org/abs/1206.0102), the problem can be relaxed to a (complex) semidefinite program (complex SDP).

Given a linear operator $A$ and a vector $b$ of measured amplitudes, define the positive semidefinite hermitian matrix $M = \text{diag}(b) (I - A A^*) \text{diag}(b)$. The problem is:

                minimize < U,M >
                subject to 
                diag(U) = 1
                U in :HermitianSemiDefinite
                
Here the variable $U$ must be hermitian ($U \in \mathbb{H}_n $), and we have a solution to the phase recovery problem if $U = u u^*$ has rank one. Otherwise, the leading singular vector of $U$ can be used to approximate the solution.


In [4]:
using Convex

n = 5
b = randn(n)
A = rand(n,n) + im*rand(n,n)
M = diagm(b)*(eye(n)-A*ctranspose(A))*diagm(b)
U = ComplexVariable(n,n)
objective = inner_product(U,M)
c1 = diag(U) == 1 
c2 = U in :SDP
p = minimize(objective,c1,c2)
solve!(p)
U.value

(size(coeff),size(var)) = ((10,10),(10,10))
----------------------------------------------------------------------------
	SCS v1.1.8 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2015
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 196
eps = 1.00e-04, alpha = 1.80, max_iters = 20000, normalize = 1, scale = 5.00
Variables n = 51, constraints m = 111
Cones:	primal zero / dual free vars: 56
	sd vars: 55, sd blks: 1
Setup time: 1.71e-04s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0|      inf       inf      -nan      -inf       inf       inf  1.57e-04 
   100| 4.05e-03  1.10e-01  5.74e-04 -9.01e+00 -9.00e+00  8.77e-18  1.20e-02 
   200| 2.19e-04  1.98e-03  1.88e-06 -9.11e+00 -9.11e+00  1.10e-16  3.61e-02 
   

5×5 Array{Complex{Float64},2}:
 1.00002-1.17067e-16im          …          0.986016-0.166703im
              1.0+0.00485926im             0.986795-0.161906im
        -0.999516+0.0314899im             -0.980259+0.197665im
        -0.971417-0.237436im              -0.997382-0.072176im
         0.986016+0.166703im       0.999993-6.0774e-17im      

In [5]:
# Verify if the rank of U is 1
A, B = eig(U.value)
length([a for a in A if(abs(real(a))>1e-4)])

1